
# sorting net

## NN that sort symbols in string

## example

Input: 

```c 
"bdcfa"
```

Output:

```c
"abcdf"
```

inspired by: 
https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py


In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [44]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
# Length of input and output string
DIGITS = 5


# All the numbers, plus sign and space for padding.
chars = 'qwertyuiopasdfghjklzxcvbnm '
ctable = CharacterTable(chars)

char_list = 'qwertyuiopasdfghjklzxcvbnm'

In [45]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: str(''.join(np.random.choice(list(char_list))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a = f()
    # Skip any addition questions we've already seen
    key = tuple(a)
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always DIGITS.
    q = '{}'.format(a)
    query = q + ' ' * (DIGITS - len(q))
    ans = ''
    for c in sorted(a):
        ans += c
    # Answers can be of maximum size DIGITS 
    ans += ' ' * (DIGITS - len(ans))
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [46]:
questions[:10]

['hh   ',
 'qnuw ',
 'okpkq',
 'ngz  ',
 'rkqrl',
 'blnyy',
 'sor  ',
 'n    ',
 'rnk  ',
 'v    ']

In [47]:
expected[:10]

['hh   ',
 'nquw ',
 'kkopq',
 'gnz  ',
 'klqrr',
 'blnyy',
 'ors  ',
 'n    ',
 'knr  ',
 'v    ']

In [48]:
print('Vectorization...')
x = np.zeros((len(questions), DIGITS, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, DIGITS)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS)

Vectorization...


In [49]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 5, 27)
(45000, 5, 27)
Validation Data:
(5000, 5, 27)
(5000, 5, 27)


In [50]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(DIGITS, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS' times as that's the maximum
# length of output.
model.add(layers.RepeatVector(DIGITS))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               79872     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 27)             3483      
_________________________________________________________________
activation_3 (Activation)    (None, 5, 27)             0         
Total params: 214,939
Trainable params: 214,939
Non-trainable params: 0
_________________________________________________________________


In [51]:
# even 10 iterations and 1 epoch is enough for 50k dataset of 5 len inputs
ITERATIONS = 10 
EPOCHS = 1

In [52]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, ITERATIONS + 1):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s - loss: 1.3539 - acc: 0.6194 - val_loss: 0.3328 - val_acc: 0.9339
Q alcgb T abcgl ☒ acdgl
Q ptv   T ptv   ☑ ptv  
Q vcz   T cvz   ☑ cvz  
Q bhl   T bhl   ☑ bhl  
Q wx    T wx    ☑ wx   
Q istow T iostw ☒ iottw
Q jzu   T juz   ☑ juz  
Q rqko  T koqr  ☑ koqr 
Q kjzlb T bjklz ☒ bjllz
Q lld   T dll   ☑ dll  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s - loss: 0.1670 - acc: 0.9772 - val_loss: 0.0839 - val_acc: 0.9940
Q yuj   T juy   ☑ juy  
Q vtvn  T ntvv  ☑ ntvv 
Q eyxy  T exyy  ☑ exyy 
Q vognu T gnouv ☑ gnouv
Q qlecd T cdelq ☒ ceelq
Q iexwv T eivwx ☑ eivwx
Q kdozd T ddkoz ☑ ddkoz
Q ltfz  T fltz  ☑ fltz 
Q dvxj  T djvx  ☑ djvx 
Q blqn  T blnq  ☑ blnq 

-----------------------------------------

In [62]:
def predict(line):
    if len(line) > DIGITS:
        print( "Length of the line {} is more then {}", len(line), DIGITS )
        return
    # pad the input
    query = line + ' ' * (DIGITS - len(line))
    x = np.zeros((1, DIGITS, len(chars)), dtype=np.bool)
    x[0] = ctable.encode(query, DIGITS)
    preds = model.predict_classes(x, verbose=0)
    guess = ctable.decode(preds[0], calc_argmax=False)
    return guess

# 

In [64]:
predict("dlc")

'cdl  '